<img src="https://cybersecurity-excellence-awards.com/wp-content/uploads/2017/06/366812.png">

<h1><center>Darwin Risk Building</center></h1>

Prior to getting started:

First, if you have just received a new api key from support, you will need to register your key and create a new user (see Register user cell)

Second, in the Environment Variables cell: 
1. Set your username and password to ensure that you're able to log in successfully
2. Set the path to the location of your datasets if you are using your own data.  The path is set for the examples.
3. Set the dataset names accordingly

Here are a few things to be mindful of:
1. For every run, check the job status (i.e. requested, failed, running, completed) and wait for job to complete before proceeding. 
2. If you're not satisfied with your model and think that Darwin can benefit from extra training, use the resume function.

Risk: Given the failure datetime, riskindex will include from 0 (healthy) to 1 (failure) with given leadtime and functional form. Note that until the riskindex becomes 1 the system has not failed yet. Riskindex can be thought as a inverse of the remaining time to the future failure.

# Set Darwin SDK

In [ ]:
from amb_sdk.sdk import DarwinSdk
ds = DarwinSdk()
ds.set_url('https://darwin-api.sparkcognition.com/v1/')

# Register user (if needed, read above)

In [ ]:
# Use only if you have a new api-key and 
# no registered users - fill in the appropriate fields then execute

#Enter your support provided api key and api key password below to register/create new users
api_key = ''
api_key_pw = ''
status, msg = ds.auth_login(api_key_pw, api_key)
if not status:
    print(msg)

#Create a new user 
status, msg = ds.auth_register_user('username', 'password','email@emailaddress.com')
if not status:
    print(msg)


# Environment Variables

In [ ]:
#Set your user id and password accordingly
USER="[your Darwin user id]"
PW="[your Darwin password]"

# Set path to datasets - The default below assumes Jupyter was started from amb-sdk/examples/Enterprise/
PATH_TO_DATASET='../../sets/'
FAILURE_DATASET='failures.csv'
TIME_SERIES_DATASET='sensor_ts.csv'

# Create a file name for your asset risk file that will be created in this workflow.  A timestamp 
# is used to create a unique name in the event you execute the workflow multiple times or with 
# different datasets.  File names must be unique in Darwin.
import datetime
ts = '{:%Y%m%d%H%M%S}'.format(datetime.datetime.now())
ASSETRISK='assetrisk-' + ts + '.csv'

# Import necessary libraries

In [ ]:
%matplotlib inline
import pandas as pd
from time import sleep

# User Login 

In [ ]:
status, msg = ds.auth_login_user(USER,PW)
if not status:
    print(msg)

# Upload data (failure data and timeseries data)

In [ ]:
# Uploads and assigns filenames to uploaded datasets
# failure date data
(code, response) = ds.upload_dataset(PATH_TO_DATASET+FAILURE_DATASET, 'unittest-failures-data')
if code == False:
    print(response)
    
# timeseries data
(code, response) = ds.upload_dataset(PATH_TO_DATASET+TIME_SERIES_DATASET, 'unittest-timeseries-data')
if code == False:
    print(response)

# Create riskindex
1. The lead time is in the unit of second (the example is for half week).
2. The lead time is the half width of the riskindex time period from 0 to 1
2. shutdown_column is the datetime when the system gets failed.
3. return_column is the datetime when the system return back healthy.
4. shutdiwn_column and return_column are in the failure dataset
5. functional_form is the risk function shape including sigmoid, step, linear, exponetial

In [ ]:
lead_time = 3600 * 24 * 7 // 2 # half week risk 
(code, response) = ds.create_risk_info('unittest-failures-data', 'unittest-timeseries-data',
                                        return_column="Date Returned to Service",
                                        shutdown_column="Shutdown Date",
                                        lead_time=lead_time, functional_form="sigmoid")
if code == False:
    print(response)
    
ds.wait_for_job(response['job_name'])

# Download the artifact (riskindex)

In [ ]:
(status, response) = ds.download_artifact(response['artifact_name'])

if code == False:
    print(response)

In [ ]:
df_risk = pd.read_csv(response['filename'])

# Read local datasets again for plotting

In [ ]:
# read timeseries data for datetime index
df_ts = pd.read_csv(PATH_TO_DATASET+TIME_SERIES_DATASET)
df_ts['datetime'] = pd.to_datetime(df_ts['datetime'], errors='coerce')

# concatenate two dfs
df = pd.concat([df_ts, df_risk], axis=1)

# set datetime index
df= df.set_index('datetime')

In [ ]:
df['risk'].plot()

In [ ]:
# see the failures
df_failure = pd.read_csv(PATH_TO_DATASET+FAILURE_DATASET)
df_failure

In [ ]:
# let's zoom-in the date around 2012-02-10 failure date
df['risk']['2012-01-25':'2012-02-12'].plot()

# Generate a full csv file and Upload for suprevised training
Now we generated the dataset which includes the risk index as a supervised label. 

In [ ]:
#View full file upon which you can build a model and convert to csv
df.to_csv(ASSETRISK)
df.head()

#Upload Full CSV to Darwin for predictions
(code, response) = ds.upload_dataset(ASSETRISK)
if code == False:
    print(response)

# Build model
Now the dataset becomes the same as supervised model.
One can consult the supervised notebook for more detail

In [ ]:
target = "risk"
model = target + "_model0" + '-' + ts
(status, job_id) = ds.create_model(dataset_names = ASSETRISK, \
                                 target = target, \
                                 model_name =  model, \
                                 max_train_time = '00:10'#,\
                                )
if status == False:
    print(response)

if status:
    ds.wait_for_job(job_id['job_name'])
else:
    print(job_id)

In [ ]:
ds.lookup_job_status_name(job_id['job_name'])

In [ ]:
#Run Predictions    
status, artifact = ds.run_model(ASSETRISK, model)
sleep(1)
ds.wait_for_job(artifact['job_name'])

In [ ]:
#Get predictions
status, prediction = ds.download_artifact(artifact['artifact_name'])
if status:
    ds.wait_for_job(job_id['job_name'])
else:
    print(job_id)

In [ ]:
#Rename
prediction = prediction.rename(columns={target:target+'_pred' })

df = df.reset_index()

# concatenate two df
df = pd.concat([df, prediction], axis=1)

# set datetime index
df= df.set_index('datetime')

In [ ]:
#Plot predictions vs actual
df[target].plot()
df[target+'_pred'].plot()

In [ ]:
# Delete all models and datasets
ds.delete_all_datasets()
ds.delete_all_models()